In [3]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import glob
import os
sys.path.insert(0,"/home/daqtest/Processor/sandpro")
import sandpro
import configparser
import json
import scipy.stats
from scipy.optimize import curve_fit
import datetime
import pandas as pd
import re
from matplotlib.pyplot import cm
import matplotlib as mpl
from dataclasses import dataclass



sys.path.insert(0,"../src/")
import common.utils as utils
import common.d2d as d2d
# import data_processing.run_info as run_info

# from run_selection_single_channel import RunInfo
# import WaveformProcessor
# import FitSPE

# from common.utils import vec_regex_search


In [4]:
params = {
    # figure
    'figure.figsize': (12, 6),
    'figure.facecolor': 'white',  # make figure background white
    # axes
    'axes.labelsize': 20,
    'axes.linewidth': 2,
    'axes.titlesize': 20,
    'axes.grid.which': 'both',  # gridlines at major, minor or both ticks
    # errorbar
    'errorbar.capsize': 4,
    # font
    'font.size': 18,
    'font.family': 'Times New Roman',
    # color
    'image.cmap': 'viridis',
    # legend
    'savefig.bbox': 'tight',
    'legend.fontsize': 20,
    'legend.frameon': False,
    'legend.numpoints': 1,  # only one marker in legend
    # line
    'lines.linestyle': 'solid',
    'lines.linewidth': 2,
    'lines.markeredgewidth': 1,
    'lines.markersize': 8,
    # text
    'mathtext.default': 'regular',
    'savefig.bbox': 'tight',
    'savefig.transparent': False,
    # tick
    'xtick.top': True,  # draw ticks on the top side
    'xtick.direction': 'in',
    'xtick.labelsize': 20,
    'xtick.major.size': 8,
    'xtick.major.width': 1,
    'xtick.minor.size': 4,
    'xtick.minor.visible': False,
    'xtick.minor.width': 1,

    'ytick.right': False,  # draw ticks on the right side
    'ytick.direction': 'in',
    'ytick.labelsize': 20,
    'ytick.major.size': 6,
    'ytick.major.width': 1,
    'ytick.minor.size': 3,
    'ytick.minor.visible': False,
    'ytick.minor.width': 1,
    # GRIDS
    'grid.linestyle': '--',  ## dashed
    'mathtext.default': 'regular',
}
plt.rcParams.update(params)


In [5]:
# @dataclass
# class run_info_data:
#     def __init__(self, input: object):
#         self.import_data(input)
    
#     def import_data(self, df: pd.DataFrame):
#         for col_name in df.columns:
#             setattr(self, col_name, df[col_name].to_numpy())
            
#     def import_data(self, dictionary: dict):
#         for var_name in dictionary.keys():
#             setattr(self, var_name, np.array(dictionary[var_name]))
            
#     def apply_mask(self, mask, inplace = False):
#         # print("before cut: ", len(self.__dict__['file_path']))
#         new_dict = {}
#         for i in self.__dict__.keys():
#             if inplace:
#                 self.__dict__[i] = self.__dict__[i][mask]
#             else:
#                 new_dict[i] = self.__dict__[i][mask]
        
#         if inplace:   
#             first = next(iter(self.__dict__.values()))
#             # print("after cut: ", len(first))
#             return
#         else:
#             first = next(iter(new_dict.values()))
#             # print("after cut: ", len(first))
#             return run_info_data(new_dict)
        
#     def get_df(self):
#         df = pd.DataFrame(self.__dict__,index=None)
#         return df
    
#     def __len__(self):
#         # the length of all array should be the same
#         # so just picked a random one
#         first = next(iter(self.__dict__.values()))
#         return len(first)
            

In [6]:
# def _re_search(pattern: str, string):
#     if pd.isnull(string):
#         return False #does not match
#     else:
#         return bool(re.search(pattern, string))
# v_regex_search = np.vectorize(_re_search, excluded=["pattern"])

In [7]:
def create_color_scheme(color_map: str, array: object, color_range=(0,1), darken=1):
    # color = cm.viridis(np.linspace(0, 1, len(np.unique(masked_temp.channel))))
    values = sorted(np.unique(array))
    
    cmap = plt.get_cmap(color_map)
    color = cmap(np.linspace(color_range[0], color_range[1], len(values)))
    
    # https://stackoverflow.com/questions/37517587/how-can-i-change-the-intensity-of-a-colormap-in-matplotlib
    color[:,0:3] *= darken
    
    clr = {values[i]: color[i] for i in range(len(values))}
    return clr

### selected_runs_gain.csv

#### Settings

In [45]:
df = pd.read_csv("/home/daqtest/DAQ/SandyAQ_vera/SandyAQ/softlink_to_data/processed/gain_info_single_channel.csv", 
                 parse_dates=["date_time"],
                 delimiter=",",
                 quotechar='"', 
                 skipinitialspace=True, 
                 encoding="utf-8")


In [ ]:
info = d2d.data(df)
print(info.__dict__.keys())

In [ ]:
color_temperature = create_color_scheme("coolwarm_r", 
                                        info.temperature_K,
                                        darken = 0.8,
                                        color_range=(0.2,0.9))
color_temperature

In [48]:
color_channel = create_color_scheme("viridis", 
                                    info.channel)

color_voltage = create_color_scheme("hot", 
                                    info.voltage_preamp1_V,
                                    color_range=(0,0.8))

#### Plots

In [ ]:
mask = ~np.isnan(info.gain)
gain_info = info.apply_mask(mask)
for temperature in np.unique(gain_info.temperature_K):
    mask = gain_info.temperature_K == temperature
    tmp = gain_info.apply_mask(mask)
    plt.errorbar(tmp.voltage_preamp1_V, tmp.gain, 
                 yerr=tmp.gain_err, 
                 label = f"{temperature} K", 
                 fmt="o", 
                 ecolor = "black", 
                 capsize=3,
                 color=color_temperature[temperature]
                 )
plt.legend()
plt.gca().invert_xaxis()

plt.xlabel("Bias Voltage [V]")
plt.ylabel("Gain")

plt.title("Gain of all channels and all datasets")


In [ ]:
voltage_list = [-46, -47, -48, -49, -50, -51, -52]

for voltage in voltage_list:
    
    fig, ax = plt.subplots()

    mask = ~np.isnan(info.gain) & (info.voltage_preamp1_V==voltage)
    gain_info = info.apply_mask(mask)

    for temperature in np.unique(gain_info.temperature_K):
        mask = gain_info.temperature_K == temperature
        tmp = gain_info.apply_mask(mask)
        ax.errorbar(tmp.date_time, tmp.gain, 
                    yerr=tmp.gain_err, 
                    label = f"{temperature} K", 
                    fmt="o", 
                    ecolor = "black", 
                    capsize=3, 
                    color=color_temperature[temperature])

    ax.legend()
    ax.tick_params(axis='x',labelrotation=30)
    ax.set_xlabel("Date Time")
    ax.set_ylabel("Gain")
    ax.set_title(f"Time evolution of gain of all channels; bias voltage at {voltage:.1f} V")


In [ ]:
voltage = -49

mask = ~np.isnan(info.gain) & (info.voltage_preamp1_V==voltage)
gain_info = info.apply_mask(mask)
# plt.errorbar(gain_info.date_time, gain_info.gain, 
#             yerr=gain_info.gain_err, 
#             label = f"{temperature} K", 
#             fmt="o", 
#             ecolor = "black", 
#             capsize=3)

    # plt.errorbar(tmp.date_time, tmp.gain, 
    #              yerr=tmp.gain_err, 
    #              label = f"{temperature} K", 
    #              fmt="o", 
    #              ecolor = "black", 
    #              capsize=3, 
    #              color=color_temperature[temperature])

for i, temperature in enumerate(np.unique(gain_info.temperature_K)):
    mask = gain_info.temperature_K==temperature
    _tmp__selection_l1 = gain_info.apply_mask(mask, inplace=False)
    for j, channel in enumerate(np.unique(gain_info.channel)):
        mask = _tmp__selection_l1.channel==channel
        _tmp__selection_l2 = _tmp__selection_l1.apply_mask(mask, inplace=False)
        color = color_temperature[temperature][:3]*((j+2)*0.05)
        # color = color_temperature[temperature][:3]*((j+3)*0.05)
        
        if i == 0:
            plt.errorbar(_tmp__selection_l2.date_time, _tmp__selection_l2.gain, 
                 yerr=_tmp__selection_l2.gain_err, 
                 label = f"{channel}", 
                 fmt="o-", 
                 ecolor = "black", 
                 capsize=3, 
                 color=color)
        else:
            plt.errorbar(_tmp__selection_l2.date_time, _tmp__selection_l2.gain, 
                 yerr=_tmp__selection_l2.gain_err, 
                 fmt="o-", 
                 ecolor = "black", 
                 capsize=3, 
                 color=color)
plt.legend()
plt.xticks(rotation=45)
plt.xlabel("Date Time")
plt.ylabel("Gain")
plt.title(f"Time evolution of gain of all channels; bias voltage at {voltage:.1f} V")


# for i, temperature in enumerate(np.unique(info.temperature_K)):
#     mask = info.temperature_K==temperature
#     _tmp__selection_l1 = info.apply_mask(mask, inplace=False)
#     for j, channel in enumerate(np.unique(info.channel)):
#         mask = _tmp__selection_l1.channel==channel
#         _tmp__selection_l2 = _tmp__selection_l1.apply_mask(mask, inplace=False)
#         color = color_temperature[temperature][:3]*((j+1)*0.05)
        
#         if i == 0:
#             plt.plot(_tmp__selection_l2.date_time, 
#                 _tmp__selection_l2.baseline_mean,"o-",
#                 color=color, label = f"{channel}")
#         else:
#             plt.plot(_tmp__selection_l2.date_time, 
#                 _tmp__selection_l2.baseline_mean,"o-",
#                 color=color)
    
plt.xlabel("Date time")
plt.ylabel("Gain")
plt.legend(bbox_to_anchor = (1,1), ncol=2)
plt.xticks(rotation=45)


In [ ]:
mask = ~np.isnan(info.gain)
gain_info = info.apply_mask(mask)

for temperature in np.unique(gain_info.temperature_K):
    gain_list = []
    gain_error_list = []
    voltage_list = []
    # print(temperature)
    mask = gain_info.temperature_K == temperature
    masked_temp = gain_info.apply_mask(mask)
    for voltage in np.unique(masked_temp.voltage_preamp1_V):
        # print(voltage)
        mask = masked_temp.voltage_preamp1_V == voltage
        masked_voltage = masked_temp.apply_mask(mask)
        if len(masked_voltage) > 0:
            gain = masked_voltage.gain.mean()
            error = masked_voltage.gain.std()
            gain_list.append(gain)
            gain_error_list.append(error)
            voltage_list.append(voltage)
            
    plt.errorbar(voltage_list, gain_list, 
                 yerr=gain_error_list, 
                 label = f"{temperature} K", 
                 fmt="o", 
                 ecolor = color_temperature[temperature], 
                 capsize=3, 
                 color=color_temperature[temperature])
    # plt.scatter(voltage_list, gain_list, label=f"{temperature} K", 
    #              color=color_temperature[temperature],
    #              edgecolor='k')
plt.legend()
plt.gca().invert_xaxis()
plt.ylabel("Gain")
plt.xlabel("Bias Voltage")
plt.title("Mean over all runs and all channels")


In [ ]:
mask = ~np.isnan(info.gain)
gain_info = info.apply_mask(mask)
for voltage in np.unique(gain_info.voltage_preamp1_V):
    mask = gain_info.voltage_preamp1_V == voltage
    tmp = gain_info.apply_mask(mask)
    gain = tmp.gain.mean()
    error = tmp.gain.std()
    plt.errorbar(voltage, gain, 
                 yerr=error, 
                 fmt="o", 
                 ecolor = "black", 
                 capsize=3,
                 markeredgecolor='k')
    # plt.scatter(voltage, gain)
plt.legend()
plt.gca().invert_xaxis()

plt.ylabel("Gain")
plt.xlabel("Bias Voltage")
plt.title("Mean over all runs, all channels, all temperature")


In [ ]:
temperature = [167, 169, 171, 173, 175]

for tempe in temperature:

    fig,ax = plt.subplots(figsize=(10,6))

    mask = ~np.isnan(info.gain)
    gain_info = info.apply_mask(mask)

    # for temperature in np.unique(gain_info.temperature_K):
    temperature = tempe

    # print(temperature)
    mask = gain_info.temperature_K == temperature
    masked_temp = gain_info.apply_mask(mask)

    for i, channel in enumerate(np.unique(masked_temp.channel)):
        gain_list = []
        gain_err_list = []
        voltage_list = []
        mask = masked_temp.channel == channel
        masked_channel = masked_temp.apply_mask(mask)
        
        for voltage in np.unique(masked_channel.voltage_preamp1_V):
            # print(voltage)
            mask = masked_channel.voltage_preamp1_V == voltage
            masked_voltage = masked_channel.apply_mask(mask)
            if len(masked_voltage) > 0:
                
                gain = masked_voltage.gain.mean()
                gain_err = np.std(masked_voltage.gain) / np.sqrt(np.size(masked_voltage.gain))
                gain_list.append(gain)
                gain_err_list.append(gain_err)
                voltage_list.append(voltage)
        
        # plt.plot(voltage_list, gain_list, 
        #         marker = "o", 
        #         label=f"{channel}", 
        #         color=color_channel[channel])
        
        plt.errorbar(voltage_list, gain_list, 
                 yerr=gain_err_list, 
                 label = f"{channel}", 
                 fmt="o-", 
                 ecolor = color_channel[channel], 
                 capsize=3,
                 color=color_channel[channel]
                 )
        
    plt.legend(bbox_to_anchor = (1,1.1),title="Channels", ncol=2)
    plt.gca().invert_xaxis()
    plt.title(f"Temperature at {temperature} K")
    plt.ylabel("Gain")
    plt.xlabel("Bias Voltage")

In [ ]:
temperature = [167, 169, 171, 173, 175]

for tempe in temperature:

    fig,ax = plt.subplots(figsize=(10,6))

    mask = ~np.isnan(info.gain)
    gain_info = info.apply_mask(mask)

    # for temperature in np.unique(gain_info.temperature_K):
    temperature = tempe

    # print(temperature)
    mask = gain_info.temperature_K == temperature
    masked_temp = gain_info.apply_mask(mask)

    for i, channel in enumerate(np.unique(masked_temp.channel)):
        gain_list = []
        gain_err_list = []
        voltage_list = []
        mask = masked_temp.channel == channel
        masked_channel = masked_temp.apply_mask(mask)
        
        for i, voltage in enumerate(np.unique(masked_channel.voltage_preamp1_V)[::-1]):
            
            mask = masked_channel.voltage_preamp1_V == voltage
            masked_voltage = masked_channel.apply_mask(mask)
            
            if len(masked_voltage) > 0:
                
                gain = masked_voltage.gain.mean()
                gain_err = np.std(masked_voltage.gain) / np.sqrt(np.size(masked_voltage.gain))
                
                
                if voltage == max(np.unique(masked_channel.voltage_preamp1_V)):
                    normalization_channel = gain
                    normalization_channel_err = gain_err
                    
                yerror = np.sqrt(gain_err**2+normalization_channel_err**2)
                
                gain_list.append(gain - normalization_channel)
                gain_err_list.append(yerror)
                voltage_list.append(voltage)
        
        # plt.plot(voltage_list, gain_list, 
        #         marker = "o", 
        #         label=f"{channel}", 
        #         color=color_channel[channel])
        
        plt.errorbar(voltage_list, gain_list, 
                 yerr=gain_err_list, 
                 label = f"{channel}", 
                 fmt="o-", 
                 ecolor = color_channel[channel], 
                 capsize=3,
                 color=color_channel[channel]
                 )
        
    plt.legend(bbox_to_anchor = (1,1.1),title="Channels", ncol=2)
    plt.gca().invert_xaxis()
    plt.title(f"Temperature at {temperature} K")
    plt.ylabel("Gain")
    plt.xlabel("Bias Voltage")

In [ ]:
# baseline and gain coorelation


fig,ax = plt.subplots(figsize=(10,6))

mask = ~np.isnan(info.gain)
gain_info = info.apply_mask(mask)
    
plt.errorbar(gain_info.baseline_mean, gain_info.gain, 
            yerr=gain_info.gain_err, 
            fmt="o", 
            capsize=3,
            )
    
plt.title(f"Correlation")
plt.xlabel("Baseline mean [V]")
plt.ylabel("Gain")

In [ ]:
# baseline and gain coorelation


fig,ax = plt.subplots(figsize=(10,6))

mask = ~np.isnan(info.gain)
gain_info = info.apply_mask(mask)

plt.errorbar(gain_info.baseline_std, gain_info.gain, 
            yerr=gain_info.gain_err, 
            fmt="o", 
            capsize=3,
            )
    
plt.title(f"Correlation")
plt.xlabel("Baseline std [V]")
plt.ylabel("Gain")

In [ ]:
# per channel evolution

voltage_list = [-46, -47, -48, -49, -50, -51, -52]

mask = ~np.isnan(info.gain)
gain_info = info.apply_mask(mask)

for voltage in voltage_list:
    
    fig, ax = plt.subplots()

    mask = (gain_info.voltage_preamp1_V==voltage)
    voltage_info = gain_info.apply_mask(mask)

    # for i, temperature in enumerate(np.unique(gain_info.temperature_K)):
    #     mask = gain_info.temperature_K==temperature
    #     _tmp__selection_l1 = gain_info.apply_mask(mask, inplace=False)
    
    for j, channel in enumerate(np.unique(voltage_info.channel)):
        mask = voltage_info.channel==channel
        channel_info = voltage_info.apply_mask(mask, inplace=False)
        
        normalization_channel = channel_info.gain[0]
        normalization_channel_err = channel_info.gain_err[0]
        
        # if i == 0:
        yerror = np.sqrt(np.square(channel_info.gain_err)+normalization_channel_err**2)
        ax.errorbar(channel_info.date_time, channel_info.gain-normalization_channel, 
            yerr=yerror, 
            label = f"{channel}", 
            fmt="o-", 
            ecolor = color_channel[channel], 
            capsize=3, 
            color= color_channel[channel])
        # else:
        #     plt.errorbar(_tmp__selection_l2.date_time, _tmp__selection_l2.gain, 
        #         yerr=_tmp__selection_l2.gain_err, 
        #         fmt="o-", 
        #         ecolor = "black", 
        #         capsize=3, 
        #         color=color)
        
    for i, tempe in enumerate(np.unique(info.temperature_K)):
        mask = gain_info.temperature_K==tempe
        temp_info = gain_info.apply_mask(mask, inplace=False)
        
        temp_date_time = sorted(temp_info.date_time)
        ax.axvspan(temp_date_time[0], temp_date_time[-1],
                   color = color_temperature[tempe],
                   alpha = 0.5)
        
    
    plt.legend(bbox_to_anchor = (1,1), ncol=2)
    plt.xticks(rotation=45)
    plt.xlabel("Date Time")
    plt.ylabel("Gain")
    plt.title(f"Time evolution of gain of all channels; bias voltage at {voltage:.1f} V")

    


In [74]:
df = pd.read_csv("/home/daqtest/DAQ/SandyAQ_vera/SandyAQ/softlink_to_data/processed/preamp1-4_correction.csv")

### Select problematic dataset

In [ ]:

# for temperature in np.unique(gain_info.temperature_K):
temperature = 169
voltage = -49
gain = 200000

mask = (info.voltage_preamp1_V == voltage)
mask = mask & (info.temperature_K == temperature)
# mask = mask & (info.channel == channel)
mask = mask & (info.gain > gain)

tmp = info.apply_mask(mask)

tmp.bin_full_path


### Threshold problem

In [ ]:
# df = pd.read_csv("../run_info_single_channel.csv", parse_dates=["date_time"],delimiter=",",quotechar='\0')
# df = pd.read_csv("../run_info_single_channel.csv", parse_dates=["date_time"],delimiter=",",quotechar='"', skipinitialspace=True, encoding="utf-8")
# df = pd.read_csv("/home/daqtest/DAQ/SandyAQ/python_wrappers/run_info_single_channel_20240701_2.csv", parse_dates=["date_time"],delimiter=",",quotechar='"', skipinitialspace=True, encoding="utf-8")
df = pd.read_csv("/home/daqtest/DAQ/SandyAQ_vera/SandyAQ/python_wrappers/run_info_single_channel_0905_2.csv", 
                 parse_dates=["date_time"],
                 delimiter=",",
                 quotechar='"', 
                 skipinitialspace=True, 
                 encoding="utf-8")
df.columns


In [ ]:
info = run_info.RunInfo(df)
print(info.__dict__.keys())

mask_record_length_nan = ~np.isnan(info.record_length_sample)
mask_run_tag = utils.vec_regex_search('GXe/gain_calibration', info.run_tag)
mask_run_tag_remove_trash = ~utils.vec_regex_search('trash', info.run_tag)
mask_time = (info.date_time > np.datetime64('2024-05-18'))
mask_start_index_nan = ~np.isnan(info.start_index)
mask_nevents_nan = ~np.isnan(info.number_of_events)

# FIXME: mask channel

mask = mask_run_tag & mask_run_tag_remove_trash & mask_time & mask_record_length_nan & mask_start_index_nan & mask_nevents_nan

# check how much data is removed
info.apply_mask(mask, inplace=True)


In [ ]:
min_bin = min(info.baseline_std)
min_bin

In [ ]:
# plt.hist2d(df.voltage_preamp1, df.baseline_std, bins=[50,50],range=[[0,100],[0,0.001]], cmap='viridis',norm="log")
min_bin = min(info.baseline_std)
max_bin = max(info.baseline_std)

# color = cm.viridis(np.linspace(0, 1, len(np.unique(info.voltage_preamp1_V))))

for i, voltage in enumerate(np.unique(info.voltage_preamp1_V)):
    mask = info.voltage_preamp1_V==voltage
    _tmp__selection = info.apply_mask(mask, inplace=False)
    # print(voltage, len(_tmp__selection))
    plt.hist(_tmp__selection.baseline_std,
             label=f"{voltage} V", 
             lw=3,
             range=(min_bin,max_bin),
             log=True,
             histtype='step',
             stacked=True, fill=False,
             color=color_voltage[voltage],
             density=True
             )
    
plt.xlabel("Voltage Preamp 1 [V]")
plt.ylabel("Baseline Std [V]")
plt.legend()



In [ ]:
fig, ax_adc = plt.subplots()

threshold_color = "tab:blue"
ax_adc.spines['left'].set_color(threshold_color)
ax_adc.tick_params(axis='y', colors=threshold_color)
ax_adc.yaxis.label.set_color(threshold_color)

color = cm.viridis(np.linspace(0, 1, len(np.unique(info.voltage_preamp1_V))))

ax_baseline = ax_adc.twinx()

# mask = (info.date_time > np.datetime64('2024-05-26'))
# mask = mask & (info.date_time < np.datetime64('2024-06-08'))
# info_time = info.apply_mask(mask, inplace=False)
info_time = info

for i, voltage in enumerate(np.unique(info.voltage_preamp1_V)):
    mask = info_time.voltage_preamp1_V==voltage
    _tmp__selection = info_time.apply_mask(mask, inplace=False)
    ax_baseline.plot(_tmp__selection.date_time, 
                _tmp__selection.baseline_std,
                'o',
                color=color_voltage[voltage],
                label = f"{voltage} V")
    
ax_adc.plot(info_time.date_time, 
            info_time.threshold_adc,'.',
            label = "threshold",
            alpha = 0.5)
    
# plt.xlabel("Date time")
# plt.ylabel("Baseline Std [V]")
ax_adc.tick_params(axis='x',labelrotation=30)
ax_adc.set_xlabel('Date Time')
ax_baseline.set_ylabel('Baseline Std [V]')
ax_adc.set_ylabel('Threshold [adc]')

plt.legend(bbox_to_anchor = (1.33,1))


In [ ]:
color = cm.viridis(np.linspace(0, 1, len(np.unique(info.temperature_K))))

# calculate the colors of each entry
# min_temp = min(info.temperature_K)
# max_temp = max(info.temperature_K)
# temp_color = (info.temperature_K - min_temp)/(max_temp-min_temp)*(len(np.unique(info.temperature_K))-1)
# temp_color.astype(int)

for i, temperature in enumerate(np.unique(info.temperature_K)):
    mask = info.temperature_K==temperature
    _tmp__selection = info.apply_mask(mask, inplace=False)
    plt.scatter(_tmp__selection.date_time, 
                _tmp__selection.baseline_std,
                color=color_temperature[temperature],
                label = f"{temperature} K")
    
plt.xlabel("Date time")
plt.ylabel("Baseline Mean [V]")
plt.legend(bbox_to_anchor = (1,1))
plt.xticks(rotation=45)

# plt.hist2d(df.voltage_preamp1, df.baseline_std, bins=[50,50],range=[[0,100],[0,0.001]], cmap='viridis',norm="log")
# plt.xlabel("Date Time")
# plt.ylabel("Baseline Mean [V]")
# plt.legend()

In [ ]:

for i, temperature in enumerate(np.unique(info.temperature_K)):
    mask = info.temperature_K==temperature
    _tmp__selection_l1 = info.apply_mask(mask, inplace=False)
    for j, channel in enumerate(np.unique(info.channel)):
        mask = _tmp__selection_l1.channel==channel
        _tmp__selection_l2 = _tmp__selection_l1.apply_mask(mask, inplace=False)
        norm = _tmp__selection_l2.baseline_mean.mean()
        color = color_temperature[temperature][:3]*((j+1)*0.05)
        
        if i == 0:
            plt.plot(_tmp__selection_l2.date_time, 
                _tmp__selection_l2.baseline_mean,".-",
                color=color, label = f"{channel}")
        else:
            plt.plot(_tmp__selection_l2.date_time, 
                _tmp__selection_l2.baseline_mean,".-",
                color=color)
    
plt.xlabel("Date time")
plt.ylabel("Baseline Mean [V]")
plt.legend(bbox_to_anchor = (1,1), ncol=2)
plt.xticks(rotation=45)


In [ ]:
mask = ~np.isnan(info.runtime_s)
mask = mask & (info.date_time > np.datetime64('2024-05-26'))
mask = mask & (info.date_time < np.datetime64('2024-06-08'))

_tmp__selection = info.apply_mask(mask,inplace=False)

# plt.hist2d(df.voltage_preamp1, df.baseline_std, bins=[50,50],range=[[0,100],[0,0.001]], cmap='viridis',norm="log")
plt.hist2d(_tmp__selection.runtime_s[:-1],
           _tmp__selection.baseline_std[1:],
           cmap='viridis', 
           bins=[100,50],
           range=[[0,2000],[0,0.05]],
           norm=mpl.colors.LogNorm())
# plt.plot(info.date_time, info.baseline_std,"o")
plt.xlabel("Run time of the previous run [s]")
plt.ylabel("Baseline Std [V]")
plt.xticks(rotation=45)
plt.legend()

### Tests

In [ ]:
# check data type all correct 
for i in info.__dict__.keys():
    print(i, type(info.__dict__[i][0]))

In [11]:

# config_name = "_".join(parts[1:4]) + ".ini"
# spd.threshold_adc = parts[3]
# spd.channel = int(parts[2])
# spd.timestamp_str = f"{int(parts[4])}_{int(parts[5].split('.')[0])}"

# spd.datetime_obj = datetime.datetime.strptime(f"{parts[4]} {parts[5].split('.')[0]}", '%Y%m%d %H%M%S')

# # ignore the channels that in the ignore_channel_list
# if spd.channel in self.ignore_channel_list: 
#     return

# # read from meta data file
# with open(meta_data, "r") as f:
#     data_taking_settings = json.load(f)

# spd.n_events = int(data_taking_settings["number_of_events"])
# spd.bias_voltage = float(data_taking_settings["voltage_config"]["preamp_1"]) # assuming all preamp has the same bias voltage; can be easily changed
# # bias_voltage_list.append(bias_voltage)


In [ ]:
nchs = 1
record_length_sample = 1000
sample_selection = 120
samples_to_average = 40

integral_window = (0.3,0.6)

gain_list = []
gain_err_list = []


mask =  (info.n_channels == nchs) & (info.record_length_sample == record_length_sample) & (info.baseline_n_samples == sample_selection) & (info.baseline_n_samples_avg == samples_to_average)
mask = mask & utils.vec_regex_search('20240625_T100_51V_5.5sig/config_6_2619_20240625_190713', info.file_path)
# check how much data is removed
print("before cut: ", len(info))
new_info = info.apply_mask(mask)
print("after cut: ", len(new_info))

process_config = {"nchs": int(nchs),
                "nsamps": int(record_length_sample),
                "sample_selection": int(sample_selection),
                "samples_to_average": int(samples_to_average)}

# dump the config to a json file
with open("process_config.json", "w") as f:
    json.dump(process_config, f, indent=4)  
    
processor= sandpro.processing.rawdata.RawData(config_file = "process_config.json",
                                            perchannel=False)

for i in range(len(new_info)):
    start_index = int(new_info.start_index[i])
    end_index = int(new_info.start_index[i] + new_info.n_processed_events[i])
    
    data = processor.get_rawdata_numpy(n_evts=int(new_info.number_of_events[i])-1,
                            file=new_info.file_path[i],
                            bit_of_daq=14,
                            headersize=4,inversion=False)
    
    wfp = WaveformProcessor.WFProcessor(os.path.dirname(new_info.file_path[i]), 
                                        volt_per_adc=2/2**14)
    wfp.set_data(data["data_per_channel"][start_index:end_index,0], in_adc = False)
    wfp.process_wfs()
    
    areas = wfp.get_area(sum_window=integral_window)
    heights = wfp.get_height(search_window=integral_window)

    data_processed = data["data_per_channel"][start_index:end_index,0,:]
    hist_count,bin_edges = np.histogram(areas,bins=200,range=(-0.1,10))

    spe_fit = FitSPE.FitSPE(info.voltage_preamp1_V[i], hist_count, bin_edges, show_plot=False, save_plot=False)
    
    if not (np.isnan(spe_fit.gain) or np.isnan(spe_fit.gain_error)):
        gain_list.append(spe_fit.gain)
        gain_err_list.append(spe_fit.gain_error)
    else:
        gain_list.append(np.nan)
        gain_err_list.append(np.nan)
    
new_info.__setattr__("gain", np.array(gain_list))
new_info.__setattr__("gain_err", np.array(gain_err_list))


In [ ]:
new_info.file_path

In [ ]:
# tag: ongoing

new_info.gain

In [ ]:
mask = ~np.isnan(new_info.gain)
tmp = new_info.apply_mask(mask)
plt.errorbar(tmp.temperature_K, tmp.gain, yerr=tmp.gain_err, fmt="ro", ecolor = "black", capsize=3)


In [ ]:
plt.errorbar(tmp.temperature_K, tmp.gain, yerr=tmp.gain_err, fmt="ro", ecolor = "black", capsize=3)

In [ ]:
len(tmp.gain_err)

In [ ]:
spe_fit.line_x

In [ ]:
mask = (info.n_channels == 1) & (info.record_length_sample == 1000) & (info.baseline_n_samples == 120) & (info.baseline_n_samples_avg == 40)

new_info = info.apply_mask(mask)


In [15]:
_tmp__selection.__setattr__("test", np.array([1,2,3,4]))

In [ ]:
n_channels_list = np.unique(info.n_channels)
record_length_list = np.unique(info.record_length_sample)
n_sample_list = np.unique(info.baseline_n_samples)
n_sample_avg_ist = np.unique(info.baseline_n_samples_avg)

integral_window = (0.3,0.6)

for nchs in n_channels_list:
    for record_length_sample in record_length_list:
        for sample_selection in n_sample_list:
            for samples_to_average in n_sample_avg_ist:
                mask =  (info.n_channels == nchs) & (info.record_length_sample == record_length_sample) & (info.baseline_n_samples == sample_selection) & (info.baseline_n_samples_avg == samples_to_average)
                # check how much data is removed
                _tmp__selection = info.apply_mask(mask)
                
                process_config = {"nchs": int(nchs),
                                "nsamps": int(record_length_sample),
                                "sample_selection": int(sample_selection),
                                "samples_to_average": int(samples_to_average)}
                
                # dump the config to a json file
                with open("process_config.json", "w") as f:
                    json.dump(process_config, f, indent=4)  
                    
                processor= sandpro.processing.rawdata.RawData(config_file = "process_config.json",
                                                            perchannel=False)
                
                for i in range(10):
                    start_index = int(_tmp__selection.start_index[i])
                    end_index = int(_tmp__selection.start_index[i] + _tmp__selection.n_processed_events[i])
                    
                    data = processor.get_rawdata_numpy(n_evts=int(_tmp__selection.number_of_events[i])-1,
                                            file=_tmp__selection.file_path[i],
                                            bit_of_daq=14,
                                            headersize=4,inversion=False)
                    
                    wfp = WaveformProcessor.WFProcessor(os.path.dirname(_tmp__selection.file_path[i]), 
                                                        volt_per_adc=2/2**14)
                    wfp.set_data(data["data_per_channel"][start_index:end_index,0], in_adc = False)
                    wfp.process_wfs()
                    
                    areas = wfp.get_area(sum_window=integral_window)
                    heights = wfp.get_height(search_window=integral_window)

                    data_processed = data["data_per_channel"][start_index:end_index,0,:]
                    hist_count,bin_edges = np.histogram(areas,bins=200,range=(-0.1,10))

                    spe_fit = FitSPE.FitSPE(hist_count, bin_edges, show_plot=False, save_plot=False)
                    
                    spe_fit.get_gain()
                    
                    
                    
                    

In [ ]:

# set the board number and integral window according to the board number (board 0 and 1 have different integral windows)
# board_number = 0
# local_channel = channel
integral_window = (0.3,0.6)
# local_channel = channel - 16


# data_file_basename = meta_data_basename.replace("meta_", "").replace(".json", f"_board_{board_number}.bin")

try:
    data = processor.get_rawdata_numpy(n_evts=df.number_of_events-1,
                                file=os.path.join(dir_name, bin_name),
                                bit_of_daq=14,
                                headersize=4,inversion=False)
    spd.start_index, spd.end_index = 2000, spd.n_events-1-500 #first 1000 events are noisy
    print(f"analysing events from range: {spd.start_index} to {spd.end_index}")
except Exception as e:
    print(e)
    data = processor.get_rawdata_numpy(1999,
                                file=os.path.join(self.data_folder, data_file_basename),
                                bit_of_daq=14,
                                headersize=4,inversion=False)

wfp = WaveformProcessor.WFProcessor(self.data_folder, volt_per_adc=2/2**14)
wfp.set_data(data["data_per_channel"][spd.start_index:spd.end_index,0], in_adc = False)
wfp.process_wfs()

spd.baseline_std = np.mean(wfp.baseline_rms)
spd.baseline_mean = np.mean(wfp.baseline)
spd.n_processed_events = len(wfp.baseline_rms)

spd.areas = wfp.get_area(sum_window=spd.integral_window)
spd.heights = wfp.get_height(search_window=spd.integral_window)

In [ ]:
# plt.hist2d(df.voltage_preamp1, df.baseline_std, bins=[50,50],range=[[0,100],[0,0.001]], cmap='viridis',norm="log")
plt.plot(df.voltage_preamp1, df.baseline_std, 'o')
plt.xlabel("Voltage Preamp 1 [V]")
plt.ylabel("Baseline Std (to be determined)")

In [ ]:
# plt.hist2d(df.voltage_preamp1, df.baseline_std, bins=[50,50],range=[[0,100],[0,0.001]], cmap='viridis',norm="log")
runtime = df.runtime.to_numpy()
baseline = df.baseline_std.to_numpy()

    
plt.xlabel("Runtime of previous file")
plt.ylabel("Baseline Std (to be determined)")

# plt.xlim(0,5000)
# bins=[200,100],range=[[-0.1,10],[0,60]],
plt.hist2d(runtime,baseline,cmap='viridis',norm="log")


In [ ]:
runtime[0] 

In [ ]:
df['file_path'] = df['file_path'].astype(str)
df['comment'] = df['comment'].astype(str)

In [ ]:
df.dtypes

In [ ]:
pd.to_datetime(df['date_time'], format="%Y%m%d_%H%M%S")

In [ ]:
# types of df columns
print(df.dtypes)